In [ ]:
import os
import torch
import torch.nn as nn
from torchvision import transforms, models
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau
from transformers import BertModel, BertTokenizer
from PIL import Image
import re

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

os.makedirs("checkpoints", exist_ok=True)

In [ ]:
# ─── Dataset Class ──────────────────────────────────────────────────
class GarbageDataset(Dataset):
    def __init__(self, root_dir, transform=None, tokenizer=None):
        self.root_dir = root_dir
        self.transform = transform
        self.tokenizer = tokenizer
        self.classes = ["Black", "Blue", "Green", "TTR"]
        self.data = []
        for label in self.classes:
            class_dir = os.path.join(root_dir, label)
            for file_name in os.listdir(class_dir):
                if file_name.endswith(".jpg") or file_name.endswith(".png"):
                    text_description = re.sub(r"\d+", "", file_name.split(".")[0])
                    self.data.append(
                        (
                            os.path.join(class_dir, file_name),
                            text_description,
                            self.classes.index(label),
                        )
                    )

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_path, text, label = self.data[idx]
        image = Image.open(img_path).convert("RGB")
        if self.transform:
            image = self.transform(image)
        if self.tokenizer:
            text = self.tokenizer(
                text,
                return_tensors="pt",
                padding="max_length",
                truncation=True,
                max_length=128,
            )
        return image, text, label

In [ ]:
# ─── Transformer Class ──────────────────────────────────────────────
transform = transforms.Compose(
    [
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
    ]
)

# ─── Tokenizer Class ────────────────────────────────────────────────
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

In [ ]:
# ─── Initiate Datasets ──────────────────────────────────────────────
train_dataset = GarbageDataset(
    root_dir="garbage_data/CVPR_2024_dataset_Train",
    transform=transform,
    tokenizer=tokenizer,
)
val_dataset = GarbageDataset(
    root_dir="garbage_data/CVPR_2024_dataset_Val",
    transform=transform,
    tokenizer=tokenizer,
)
test_dataset = GarbageDataset(
    root_dir="garbage_data/CVPR_2024_dataset_Test",
    transform=transform,
    tokenizer=tokenizer,
)

# ─── Initiate Dataloaders ───────────────────────────────────────────
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

In [ ]:
# ─── Model Class ────────────────────────────────────────────────────
class MultimodalModel(nn.Module):
    def __init__(self, num_classes=4):
        super(MultimodalModel, self).__init__()
        # ─── Image Model ────────────────────────────────────
        self.image_model = models.resnet50(pretrained=True)
        num_ftrs = self.image_model.fc.in_features
        self.image_model.fc = nn.Identity()

        # ─── Text Model ─────────────────────────────────────
        self.text_model = BertModel.from_pretrained("bert-base-uncased")

        # ─── Combined Classifiers With Hidden Layers ────────
        self.fc1 = nn.Linear(num_ftrs + self.text_model.config.hidden_size, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, num_classes)
        self.sigmoid = nn.Sigmoid()
        self.dropout = nn.Dropout(0.2)

    def forward(self, image, text):
        # ─── Image Features ─────────────────────────────────
        image_features = self.image_model(image)

        # ─── Text Features ──────────────────────────────────
        text_features = self.text_model(**text).last_hidden_state[:, 0, :]

        # ─── Combine Features ───────────────────────────────
        combined_features = torch.cat((image_features, text_features), dim=1)

        # ─── Pass Through Additional Hidden Layers ──────────
        x = self.fc1(combined_features)
        x = self.sigmoid(x)
        x = self.dropout(x)
        x = self.fc2(x)
        x = self.sigmoid(x)
        x = self.dropout(x)
        x = self.fc3(x)
        output = self.sigmoid(x)

        return output

In [ ]:
# ─── Initiate Model ─────────────────────────────────────────────────
model = MultimodalModel(num_classes=4)
model.to(device)

In [ ]:
# ─── Criterion ──────────────────────────────────────────────────────
criterion = nn.CrossEntropyLoss()
# ─── Optimizer ──────────────────────────────────────────────────────
optimizer = optim.Adam(model.parameters(), lr=1e-4)
# ─── Scheduler ──────────────────────────────────────────────────────
scheduler = ReduceLROnPlateau(optimizer, "min", patience=3, factor=0.1, verbose=True)